In [1]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from xgboost import XGBRegressor

from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
prject_path = "/Users/richardxu/Dropbox/UIUC_CS598_Statistical_Learning/Project2/Proj2_Data"
fold_num = 1
fold = "fold_" + str(fold_num)
df_train = pd.read_csv(os.path.join(prject_path, fold ,"train.csv"), parse_dates=['Date'])
df_test = pd.read_csv(os.path.join(prject_path, fold, "test.csv"), parse_dates=['Date'])

### Step1: transform Date to Year/Week

In [3]:
def transform_train_data_with_year_week(dataframe: pd.DataFrame):
    new_df = dataframe.copy()
    new_df['Year'] = new_df['Date'].dt.isocalendar().year
    new_df['Week'] = new_df['Date'].dt.isocalendar().week
    new_df.drop(columns=['Date','IsHoliday'], inplace=True)
    
    # combine year and week for svd smoothing
    new_df['Year_Week'] = new_df['Year'] * 100 + new_df['Week']
    new_df.drop(columns=['Year','Week'], inplace=True)

    # Re-order column names
    return new_df[['Store','Dept','Year_Week','Weekly_Sales']]

In [4]:
def transform_test_data_with_year_week(dataframe: pd.DataFrame):
    new_df = dataframe.copy()
    new_df['Year'] = new_df['Date'].dt.isocalendar().year
    new_df['Week'] = new_df['Date'].dt.isocalendar().week
    new_df.drop(columns=['Date'], inplace=True)

    # Re-order column names
    return new_df[['Store','Dept', 'Year', 'Week', 'IsHoliday']]

In [5]:
df_train_transformed = transform_train_data_with_year_week(df_train)

In [6]:
df_test_transform = transform_test_data_with_year_week(df_test)
#df_test_transform

### Step2: Group data based on (Store, Dept) pairs

In [7]:
def group_data(dateframe: pd.DataFrame,
               keys: list= ['Store', 'Dept']):
    df_grouped = dateframe.groupby(keys)
    group_ids = []
    groups = []
    for id in df_grouped.groups:
        group_ids.append(id)
        groups.append(df_grouped.get_group(id).drop(columns=keys))
    
    return group_ids, groups

### Step3: SVD smoothing on individual groups, remove outliers/noise, filling missing values

In [8]:
def smoothing_train_data_with_svd(dataframe: pd.DataFrame,
                                  num_pc: int = 8):
    ## fill out missing values/clean outliers
    group_ids, groups = group_data(dataframe, keys=['Dept'])
    
    svd_smoothed_df_list = []
    for i in range(len(group_ids)):    
        df_i = groups[i].pivot(index='Store', columns='Year_Week', values='Weekly_Sales').fillna(0)
        mean_i = df_i.mean(axis=1).values
        df_i_values_centered = (df_i.values.T - mean_i).T
        U, S, Vh = np.linalg.svd(df_i_values_centered, full_matrices=False)
        new_S = np.diag(S[:num_pc])

        df_svd_smoothed_values = ((U[:,:num_pc]@new_S@Vh[:num_pc, :]).T + mean_i).T
        

        df_svd_smoothed = pd.DataFrame(df_svd_smoothed_values,
                                       index=df_i.index,
                                       columns=df_i.columns).reset_index()

        df_i_svd_smoothed_unpivot = pd.melt(df_svd_smoothed, 
                                            id_vars='Store',
                                            value_vars=df_svd_smoothed.columns).\
                                    sort_values(by=['Store','Year_Week'])
        
        df_i_svd_smoothed_unpivot['Year'] = (df_i_svd_smoothed_unpivot['Year_Week'].values//100).astype(int)
        df_i_svd_smoothed_unpivot['Week'] = (df_i_svd_smoothed_unpivot['Year_Week'].values%100).astype(int)
        df_i_svd_smoothed_unpivot.rename(columns={'value':'Weekly_Sales'}, inplace=True)
        df_i_svd_smoothed_unpivot.drop(columns=['Year_Week'], inplace=True)
        df_i_svd_smoothed_unpivot.reset_index(drop=True)
        df_i_svd_smoothed_unpivot['Dept'] = group_ids[i]
        svd_smoothed_df_list.append(df_i_svd_smoothed_unpivot)

    
    svd_smoothed_df = pd.concat(svd_smoothed_df_list).reset_index(drop=True)
    holiday_weeks = [6, 36, 47, 52]
    svd_smoothed_df['IsHoliday'] = np.where(svd_smoothed_df['Week'].isin(holiday_weeks), True, False)
    
    # Keep column order consistent with original one
    return svd_smoothed_df[['Store', 'Dept','Year', 'Week', "IsHoliday", 'Weekly_Sales']]

        

In [9]:
df_train_svd_smoothed =  smoothing_train_data_with_svd(dataframe=df_train_transformed)

### Step4: one-hot encoding

In [10]:
df_train_svd_smoothed['Week'] = df_train_svd_smoothed['Week'].astype('object')
df_train_svd_smoothed['IsHoliday'] = df_train_svd_smoothed['IsHoliday'].astype('object')

In [11]:
def categorical_variable_transform(train_df, test_df):
    # IMPORTANT:
    # The test_dataframe needs to use the encoder from the trainng_dataframe, because some categories might be
    # missing in the test data
    
    categorical_feature_set = [feature for feature in train_df.columns if train_df[feature].dtypes=='object']
    new_train_df = train_df.copy()
    new_test_df = test_df.copy()
    for feature in categorical_feature_set:
        encoder = OneHotEncoder(handle_unknown='ignore')
        train_category_matrix = [[element] for element in train_df[feature]]
        test_category_matrix = [[element] for element in test_df[feature]]

        encoder.fit(train_category_matrix)
        train_df_hot_code = pd.DataFrame(encoder.transform(train_category_matrix).toarray())
        test_df_hot_code = pd.DataFrame(encoder.transform(test_category_matrix).toarray())

        # Different from Project#1, add 1 here
        train_df_hot_code.columns = [feature + '_' + str(c+1) for c in train_df_hot_code.columns]
        test_df_hot_code.columns = [feature + '_' + str(c+1) for c in test_df_hot_code.columns]


        # Replace the original feature with one-hot encoded feature
        new_train_df.drop(columns=feature, inplace=True)
        new_train_df = pd.concat([new_train_df, train_df_hot_code], axis=1)
        
        new_test_df.drop(columns=feature, inplace=True)
        new_test_df = pd.concat([new_test_df, test_df_hot_code], axis=1)

    # Further feature engineering
    new_train_df.drop(columns='Weekly_Sales', inplace=True)
    new_train_df['Year'] = new_train_df['Year'] - 2010 

    new_test_df['Year'] = new_test_df['Year'] - 2010

    return new_train_df, new_test_df

In [12]:
df_train_one_hot_encode, df_test_one_hot_encode = categorical_variable_transform(df_train_svd_smoothed, df_test_transform)

### Step 5. Build Lasso regression models and make predictions for individual groups

In [13]:
# Y : Weekly_Sales
# Each group has weekly_sales associated with a (Store, Dept) pair
train_Y_group_ids, train_Y_groups = group_data(df_train_svd_smoothed[['Store', 'Dept', 'Weekly_Sales']])

In [14]:
# Group data by (Store, Dept) pair
train_X_group_ids, train_X_groups = group_data(df_train_one_hot_encode, keys= ['Store', 'Dept'])
test_X_group_ids, test_X_groups = group_data(df_test_one_hot_encode, keys=['Store', 'Dept'])

In [15]:
prediction_results = []
for i, index_pair in enumerate(test_X_group_ids):
    if index_pair not in train_X_group_ids:
        #print("Oops, cannot find at ({}, {})".format(index_pair[0], index_pair[1]))
        pred_test = np.zeros(len(test_X_groups[i]))
    else:
        i_train = train_X_group_ids.index(index_pair)

        temp_train_X = train_X_groups[i_train]
        temp_train_Y = train_Y_groups[i_train]
        temp_test_X = test_X_groups[i]
        
        # Not working as good as Lasso
        #xgb_model = XGBRegressor(n_estimators=100,max_depth=2)
        #xgb_pipeline = Pipeline(steps=[("scalar",StandardScaler()), ("xgb", xgb_model)])
        #xgb_pipeline.fit(temp_train_X, temp_train_Y)
        #pred_test = xgb_pipeline.predict(temp_test_X)
        

        # The training X are basically 1 and 0 s, no need to stanardize

        lasso_model = Lasso(alpha = 10)
        #lasso_pipeline = Pipeline(steps=[("scalar",StandardScaler()), ("lasso", lasso_model)])
        #lasso_pipeline.fit(temp_train_X, temp_train_Y)
        #pred_test = lasso_pipeline.predict(temp_test_X)
        lasso_model.fit(temp_train_X, temp_train_Y)
        pred_test = lasso_model.predict(temp_test_X)

    prediction_results.extend(list(pred_test))

/Users/richardxu/opt/anaconda3/envs/python39/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(


In [16]:
df_test['Weekly_Pred'] = np.array(prediction_results)

### Step 6: Evaluate performance

In [17]:
df_test_with_label = pd.read_csv(os.path.join(prject_path, "test_with_label.csv"), parse_dates=['Date'])

In [18]:
scoring_df = df_test.drop(columns=['IsHoliday']).merge(df_test_with_label, on=['Store', 'Dept','Date'], how='left')

In [19]:
# extract weights and convert to numpy arrays for wae calculation
weights = scoring_df['IsHoliday'].apply(lambda x:5 if x else 1)
actuals = scoring_df['Weekly_Sales']
preds = scoring_df['Weekly_Pred']

wae = np.sum(weights * np.abs(actuals - preds)) / np.sum(weights)

In [20]:
wae

1820.752145237265

### Manualy run across 10 folders, got an average of 1590

In [21]:
(1820+1359.5+1406.4+1482.3+2290+1633.8+1678.6+1396.4+1418.08+1420)/10

1590.508